### Import various libraries and packages

In [ ]:
from afft.utils.log import logger

### Create SQL engine
Create a SQL engine with SQLalchemy. Note that the engine has not connected to the database yet.

In [ ]:
import sqlalchemy as db

from dotenv import dotenv_values
from afft.utils.log import logger


values = dotenv_values("/home/martin/dev/afft/.env")


url: db.engine.URL = db.engine.URL.create(
    drivername="postgresql",
    database="acfr_auv_messages",
    host="localhost",
    port=5432,
    username=values.get("PG_USER"),
    password=values.get("PG_PASSWORD"),
)

logger.info(f"Creating engine: {url}")

engine: db.Engine = db.create_engine(url)

### Connect to database
To start interacting with the database you can be explicit and use the connect method.

In [ ]:
from sqlalchemy.exc import OperationalError

if False:
    try:
        engine.connect()
    except (OperationalError, OSError, IOError) as error:
        logger.error(error)

### Read and parse messages

In [ ]:
from pathlib import Path

from afft.io import read_config, read_lines
from afft.services.sirius import Message
from afft.services.sirius import (
    MessageProtocol,
    build_message_protocol,
    parse_message_lines,
)
from afft.utils.log import logger


def parse_messages(source: str, config: str) -> dict[str, Message]:
    """Reads message lines from a source and parses the messages."""

    config: dict = read_config(Path(config)).unwrap()
    lines: list[str] = read_lines(Path(source)).unwrap()

    protocol: MessageProtocol = build_message_protocol(config.get("message_maps"))
    parsed_messages: dict[str, Message] = parse_message_lines(lines, protocol)

    return parsed_messages


MESSAGE_DIR: Path = Path(
    "/home/martin/data/acfr_revisits_messages/acfr_merged_messages"
)
MESSAGE_FILE: Path = MESSAGE_DIR / Path("r23685bc_20100605_021022_messages.txt")
PROTOCOL_FILE: Path = Path("/home/martin/dev/afft/config/protocol/protocol_v1.toml")

parsed_messages: dict[str, Message] = parse_messages(MESSAGE_FILE, PROTOCOL_FILE)

for topic, messages in parsed_messages.items():
    logger.info(f"Topic: {topic}, message count: {len(messages)}")

### Test

In [ ]:
for topic, messages in parsed_messages.items():
    data = messages[0].header.to_dict()
    data.update(messages[0].body.to_dict())

    logger.info(data)
    logger.info(messages[0].to_dict())

### Convert dataclasses to data frames with polars

In [ ]:
import polars as pl

dataframes: dict[str, pl.DataFrame] = dict()

for topic, messages in parsed_messages.items():
    # Convert dataclasses to dictionaries
    data: list[dict] = [message.to_dict() for message in messages]

    # Convert dictionaries to dataframe
    dataframes[topic] = pl.DataFrame(data)

### Create table and ingest dataframes with polars

In [ ]:
with engine.begin() as connection:
    for topic, dataframe in dataframes.items():
        status: int = dataframe.write_database(
            table_name=topic,
            connection=connection,
            # connection: ConnectionOrCursor | str,
            # if_table_exists: DbWriteMode = 'fail',
            # engine: DbWriteEngine | None = None,
            # engine_options: dict[str, Any] | None = None,
        )

        logger.info(f"Database write - topic: {topic}, status: {status}")